# Momentum XGBoost


First thing we need to do is; <br>
Check literature for asset allocation <br>
Then; <br>
Get data <br>
Clean dataset and format it to match model <br>
Build the model and run the regression <br>
Implement it to a system <br>


In [1]:
#!pip install pandas==1.1.5

In [2]:
import sys
import time
from time import strftime
import datetime
from calendar import monthrange
import os

import numpy as np
import pandas as pd
import pandas_datareader as pdr
 
import sklearn
from sklearn.linear_model import Lasso, Ridge, ElasticNet, LinearRegression
from sklearn.model_selection import cross_val_score, train_test_split, KFold, GridSearchCV

import xgboost

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
sns.set_style('whitegrid')
%matplotlib inline

random_state=42
np.random.seed(random_state)



In [3]:
#!pip3 install cmake


In [4]:
table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df = table[0]
df


,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
3,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
4,ACN,Accenture,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...,...
500,YUM,Yum! Brands Inc,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
501,ZBRA,Zebra Technologies,reports,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
502,ZBH,Zimmer Biomet,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
503,ZION,Zions Bancorp,reports,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


In [5]:
df = df[['Symbol', 'GICS Sector']]

In [6]:
type(df['GICS Sector'][1])

str

In [7]:
df['True/False'] = df['GICS Sector'] == 'Information Technology'

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [8]:
df['True/False'] = df['True/False'].astype(str)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [9]:
df = df[df["True/False"].str.contains('True')]

In [10]:
df.drop('GICS Sector', axis=1, inplace=True)

In [11]:
df.drop('True/False', axis=1, inplace=True)

In [12]:
df

,Symbol
4,ACN
6,ADBE
7,AMD
13,AKAM
38,APH
...,...
475,V
489,WDC
490,WU
498,XLNX


In [13]:
tickers = list(df['Symbol'])

In [14]:
import datetime as dt
start = dt.datetime(2010, 1, 1)
end = dt.datetime(2021, 1, 1)
data = pdr.DataReader(tickers, 'yahoo', start, end) ['Adj Close']
data

Symbols,ACN,ADBE,AMD,AKAM,APH,ADI,ANSS,AAPL,AMAT,ANET,...,TER,TXN,TRMB,TYL,VRSN,V,WDC,WU,XLNX,ZBRA
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,33.718582,37.090000,9.700000,25.920000,10.532363,23.794266,44.639999,6.583586,11.493038,NaN,...,10.287004,19.490353,12.825000,20.299999,21.007998,20.297867,35.536190,13.546346,19.830650,28.670000
2010-01-05,33.926979,37.700001,9.710000,26.690001,10.312269,23.756701,43.639999,6.594968,11.404632,NaN,...,10.324549,19.377947,12.760000,20.780001,21.228777,20.065269,36.478378,13.489576,19.580618,28.620001
2010-01-06,34.287651,37.619999,9.570000,26.469999,10.351247,23.711624,43.349998,6.490066,11.380518,NaN,...,10.230688,19.235573,12.725000,20.770000,20.999506,19.795835,36.509796,13.482479,19.447794,28.400000
2010-01-07,34.255581,36.889999,9.470000,26.480000,10.282465,23.523796,43.459999,6.478067,11.259960,NaN,...,10.287004,19.295523,12.745000,20.500000,20.710794,19.980068,35.795292,13.915334,19.252451,27.690001
2010-01-08,34.119335,36.689999,9.430000,26.480000,10.358120,23.659035,43.759998,6.521136,11.693967,NaN,...,10.399633,19.737629,12.845000,20.740000,20.872135,20.035326,35.975883,14.050159,19.533739,27.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-24,256.423737,499.859985,91.809998,107.459999,64.439354,143.853745,364.170013,131.773087,85.168358,288.839996,...,118.794655,160.665756,66.650002,434.260010,214.589996,208.376495,49.919998,21.655905,141.990005,376.890015
2020-12-28,258.795624,498.950012,91.599998,107.279999,64.773598,144.062851,363.720001,136.486053,84.709236,290.940002,...,117.645485,160.496780,66.430000,431.929993,216.800003,212.300415,50.060001,21.507303,141.520004,382.589996
2020-12-29,256.882202,502.109985,90.620003,106.709999,64.075180,143.047211,360.850006,134.668762,84.110367,287.500000,...,117.115860,160.208511,65.769997,429.369995,215.630005,214.037704,48.290001,21.378517,139.899994,378.040009


In [15]:
#dropping values which is nan
data.dropna(axis=1, inplace=True)

In [16]:
data.to_csv('momentum.csv')

In [17]:
data_momentum_monthly = data.resample('M').last()

In [18]:
data_momentum=pd.DataFrame()
lags = [1,2,3,6,9,12]
for lag in lags:
    data_momentum[f'return_{lag}m'] = (data_momentum_monthly.pct_change(lag).stack().add(1).pow(1/lag).sub(1))

In [19]:
data_momentum.dropna(axis=0, inplace=True)

In [20]:
data_momentum.to_csv('data_momentum.csv')